<a href="https://colab.research.google.com/github/marleyrosario/artificial-intelligence/blob/main/facial_recognition_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface
from deepface import DeepFace

In [5]:
import pandas as pd
import urllib
import requests
import os

In [ ]:
path = 'Select * FROM `people-first-337420.PB_lists.comments_juv`'
project_id = """people-first-337420"""

df = pd.read_gbq(query = path, project_id = project_id)

In [ ]:
print(df['profilePictureUrl'])

In [8]:
df['profilePictureUrl'] = df['profilePictureUrl'].astype("string")
df = df[df['profilePictureUrl'].str.contains('http')]
df['profilePictureUrl'] = df['profilePictureUrl'] + '.jpg'
profile_pictures = df['profilePictureUrl'].to_list()
df['index1'] = df.index + 1
df['index1'] = "image" + df['index1'].astype(str) + ".jpg"


In [9]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'people-first-337420'
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

# Generate a random bucket name to which we'll upload the file.
import uuid
bucket_name = 'commenters_juv_image_analysis' 

#body = {
 # 'name': bucket_name,
  # For a full list of locations, see:
  # https://cloud.google.com/storage/docs/bucket-locations
#  'location': 'us',
#}
#gcs_service.buckets().insert(project=project_id, body=body).execute()
#print('Done')

In [ ]:
from google.colab import files as FILE
import os
from google.colab import drive
from googleapiclient.http import MediaFileUpload

def save_images_to_bucket(file, count):
#file = 'https://scontent-lga3-2.cdninstagram.com/v/t51.2885-19/271230313_1074620846654159_9048096372490609200_n.jpg?stp=dst-jpg_s150x150&_nc_ht=scontent-lga3-2.cdninstagram.com&_nc_cat=111&_nc_ohc=SF1WdWibkqAAX97BrsD&edm=AId3EpQBAAAA&ccb=7-4&oh=00_AT_g1j6y3uEMUnGyJWplbgw7VvOqfQiyravE0QJSZ2vqow&oe=627F5085&_nc_sid=705020.jpg'
  img_data = requests.get(file).content
  with open(f'/tmp/image_{count}.jpg', 'wb') as handler:
        handler.write(img_data)



  media = MediaFileUpload(f'/tmp/image_{count}.jpg', 
                        mimetype='text/plain',
                        resumable=True)

  request = gcs_service.objects().insert(bucket=bucket_name, 
                                        name=f'image_{count}.jpg',
                                        media_body=media)

  response = None
  while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
      _, response = request.next_chunk()

  print('Upload complete')



In [ ]:
count = 0
for i in profile_pictures:
  save_images_to_bucket(i, count)
  count = count +1

#print('https://console.cloud.google.com/storage/browser?project={}'.format(project_id))

In [10]:
from apiclient.http import MediaIoBaseDownload
bucket_name = 'commenters_juv_image_analysis' 
with open('/tmp/gsutil_download.txt', 'wb') as f:
  # Download the file from a given Google Cloud Storage bucket.
  request = gcs_service.objects().get_media(bucket=bucket_name,
                                            object='image_1.jpg')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()
  print('Download complete')
  

Download complete


In [ ]:
!cat /tmp/gsutil_download.txt

In [11]:
def run_facial_rec(fname):
    bucket_name = 'commenters_juv_image_analysis' 
    with open('/tmp/gsutil_download.txt', 'wb') as f:
  # Download the file from a given Google Cloud Storage bucket.
      request = gcs_service.objects().get_media(bucket=bucket_name,
                                                object=f'{fname}')
      media = MediaIoBaseDownload(f, request)

      done = False
      while not done:
        # _ is a placeholder for a progress object that we ignore.
        # (Our file is small, so we skip reporting progress.)
        _, done = media.next_chunk()
    print('Download complete')
    face = DeepFace.analyze(img_path = f'/tmp/{fname}', actions = ['age', 'gender', 'race', 'emotion'], enforce_detection = False)
    return face


fname = "image"
list_of_fnames = []
for i in range(len(profile_pictures)+1):
     list_of_fnames.append(fname + "_" +f"{i}")
        
list_of_fnames = [fname + ".jpg" for fname in list_of_fnames]

In [12]:
print(list_of_fnames)

['image_0.jpg', 'image_1.jpg', 'image_2.jpg', 'image_3.jpg', 'image_4.jpg', 'image_5.jpg', 'image_6.jpg', 'image_7.jpg', 'image_8.jpg', 'image_9.jpg', 'image_10.jpg', 'image_11.jpg', 'image_12.jpg', 'image_13.jpg', 'image_14.jpg', 'image_15.jpg', 'image_16.jpg', 'image_17.jpg', 'image_18.jpg', 'image_19.jpg', 'image_20.jpg', 'image_21.jpg', 'image_22.jpg', 'image_23.jpg', 'image_24.jpg', 'image_25.jpg', 'image_26.jpg', 'image_27.jpg', 'image_28.jpg', 'image_29.jpg', 'image_30.jpg', 'image_31.jpg', 'image_32.jpg', 'image_33.jpg', 'image_34.jpg', 'image_35.jpg', 'image_36.jpg', 'image_37.jpg', 'image_38.jpg', 'image_39.jpg', 'image_40.jpg', 'image_41.jpg', 'image_42.jpg', 'image_43.jpg', 'image_44.jpg', 'image_45.jpg', 'image_46.jpg', 'image_47.jpg', 'image_48.jpg', 'image_49.jpg', 'image_50.jpg', 'image_51.jpg', 'image_52.jpg', 'image_53.jpg', 'image_54.jpg', 'image_55.jpg', 'image_56.jpg', 'image_57.jpg', 'image_58.jpg', 'image_59.jpg', 'image_60.jpg', 'image_61.jpg', 'image_62.jpg', '

In [ ]:
x = []
for fname in list_of_fnames:
    try:  
        results = [run_facial_rec(fname = fname)]
        x.append(results)        
    except Exception as e: 
        x.append(fname)
        print(e)

In [14]:
print(x)

['image_0.jpg', 'image_1.jpg', 'image_2.jpg', 'image_3.jpg', 'image_4.jpg', 'image_5.jpg', 'image_6.jpg', 'image_7.jpg', 'image_8.jpg', 'image_9.jpg', 'image_10.jpg', 'image_11.jpg', 'image_12.jpg', 'image_13.jpg', 'image_14.jpg', 'image_15.jpg', 'image_16.jpg', 'image_17.jpg', 'image_18.jpg', 'image_19.jpg', 'image_20.jpg', 'image_21.jpg', 'image_22.jpg', 'image_23.jpg', 'image_24.jpg', 'image_25.jpg', 'image_26.jpg', 'image_27.jpg', 'image_28.jpg', 'image_29.jpg', 'image_30.jpg', 'image_31.jpg', 'image_32.jpg', 'image_33.jpg', 'image_34.jpg', 'image_35.jpg', 'image_36.jpg', 'image_37.jpg', 'image_38.jpg', 'image_39.jpg', 'image_40.jpg', 'image_41.jpg', 'image_42.jpg', 'image_43.jpg', 'image_44.jpg', 'image_45.jpg', 'image_46.jpg', 'image_47.jpg', 'image_48.jpg', 'image_49.jpg', 'image_50.jpg', 'image_51.jpg', 'image_52.jpg', 'image_53.jpg', 'image_54.jpg', 'image_55.jpg', 'image_56.jpg', 'image_57.jpg', 'image_58.jpg', 'image_59.jpg', 'image_60.jpg', 'image_61.jpg', 'image_62.jpg', '

In [ ]:
fr_results = pd.DataFrame({"fr_results": x, "index1": list_of_fnames})

len(fr_results)

25000

In [ ]:


number =[{k:v for k,v in result.items() if k.startswith('r')} for result in y] 
race = [{k:v for k,v in result.items() if "dominant_race" in k} for result in y] 
age = [{k:v for k,v in result.items() if "age" in k} for result in y] 
gender = [{k:v for k,v in result.items() if "gender" in k} for result in y]


number = [nonsense['race'] for nonsense in number]
#numbers = df['number'].tolist()

race = [ sub['dominant_race'] for sub in race]
#races = df['race'].tolist()

age = [ nonsense['age'] for nonsense in age]
#ages = df['age'].tolist()

gender = [ thing['gender'] for thing in gender]
#genders = df['gender'].tolist()


In [ ]:
dict = {'dominant_race' : race, 'age': age, 'gender': gender, 'race_breakdown': number}
df = pd.DataFrame(dict)
df.to_csv("MBFollowerDemographicData.csv")

In [ ]:
#df = fr_results.merge(df, on = 'index1')
#df.to_csv("MBFollowerDemographicData_2.csv")

len(df)

24999